# Deliverables:

- Submit a single zip-compressed file that has the name: YourLastName_Exercise_1 that has the following files:

 1. Your **PDF document** that has your Source code and output
 2. Your **ipynb script** that has your Source code and output


# Objectives:

In this exercise, you will:

 - Construct hierarchical indexes
 - Select and group data to create pivot-tables




# Submission Formats :

Create a folder or directory with all supplementary files with your last name at the beginning of the folder name, compress that folder with zip compression, and post the zip-archived folder under the assignment link in Canvas. The following files should be included in an archive folder/directory that is uploaded as a single zip-compressed file. (Use zip, not StuffIt or any 7z or any other compression method.)


1. Complete IPYNB script that has the source code in Python used to access and analyze the data. The code should be submitted as an IPYNB script that can be be loaded and run in Jupyter Notebook for Python
2. Output from the program, such as console listing/logs, text files, and graphics output for visualizations. If you use the Data Science Computing Cluster or School of Professional Studies database servers or systems, include Linux logs of your sessions as plain text files. Linux logs may be generated by using the script process at the beginning of your session, as demonstrated in tutorial handouts for the DSCC servers.
3. List file names and descriptions of files in the zip-compressed folder/directory.


Formatting Python Code
When programming in Python, refer to Kenneth Reitz’ PEP 8: The Style Guide for Python Code:
http://pep8.org/ (Links to an external site.)Links to an external site.
There is the Google style guide for Python at
https://google.github.io/styleguide/pyguide.html (Links to an external site.)Links to an external site.
Comment often and in detail.


# Specifications and Requirements


We're going to use the XYZ data again  to construct hierarchical indexes and select, modify, group, and reshape data in a wide variety of ways.  The data we want here, which we'll call xyzcustnew, are as follows:

In [1]:

import pandas as pd  # panda's nickname is pd
import numpy as np  # numpy as np
from pandas import DataFrame, Series, Categorical 
from sqlalchemy import create_engine


In [2]:
engine=create_engine('sqlite:///xyz.db')	    # the db is in my current working directory

In [3]:
xyzcustnew=pd.read_sql_table('xyzcust',engine)


In [4]:
# Refer to exercise #7 how we calculated this value for xyz db
heavyCut= 423  #heavyCut is a constant

In [6]:
heavyCat=Categorical(np.where(xyzcustnew.YTD_SALES_2009>heavyCut,1,0))
heavyCat.describe()

,counts,freqs
categories,,
0,25795,0.854733
1,4384,0.145267


In [8]:
heavyCat.rename_categories(['regular','heavy'],inplace=True)

In [9]:
heavyCat.describe()

,counts,freqs
categories,,
regular,25795,0.854733
heavy,4384,0.145267


In [10]:
heavyCat[:10]

[regular, heavy, regular, regular, regular, regular, heavy, regular, regular, regular]
Categories (2, object): [regular, heavy]

In [11]:
xyzcustnew['heavyCat']=heavyCat

In [12]:
buyerType=pd.get_dummies(heavyCat)

In [13]:
buyerType[:3]

,regular,heavy
0,1,0
1,0,1
2,1,0


In [14]:
xyzcustnew['typeReg']=buyerType['regular']
xyzcustnew['typeHeavy']=buyerType['heavy']


In [15]:
xyzcustnew.columns

Index(['index', 'ACCTNO', 'ZIP', 'ZIP4', 'LTD_SALES', 'LTD_TRANSACTIONS',
       'YTD_SALES_2009', 'YTD_TRANSACTIONS_2009', 'CHANNEL_ACQUISITION',
       'BUYER_STATUS', 'ZIP9_SUPERCODE', 'heavyCat', 'typeReg', 'typeHeavy'],
      dtype='object')

In [16]:
# for this exercises we need to create trCountsChrono object similar to what we did in exercises #8

xyztrans=pd.read_sql('xyztrans', engine)

trandate=xyztrans.TRANDATE	# should be a Series

daystr=trandate.str[0:2]		# two digit date numbers slice

mostr=trandate.str[2:5]		# the three letter month abbreviations

yearstr=trandate.str[5:]		# four digit years


In [17]:
#create a dictionary for the months
monums={'JAN':'1', 'FEB':'2', 'MAR':'3', 'APR':'4', 'MAY':'5', 'JUN':'6', 'JUL':'7', 'AUG':'8', 'SEP':'9', 'OCT':'10', 'NOV':'11','DEC':'12'}
#month
monos=mostr.map(monums)	# do a dict lookup for each value of mostr

transtr=yearstr+'-'+monos+'-'+daystr

In [18]:
trDateTime=pd.to_datetime(transtr)

In [19]:

trCounts=trDateTime.value_counts()

In [20]:
newIndex=pd.date_range(trCounts.index.min(),trCounts.index.max())

trCountsChrono=trCounts.reindex(index=newIndex)


In [21]:
print(trCountsChrono.head())

2009-01-01    176
2009-01-02    305
2009-01-03    365
2009-01-04    231
2009-01-05    144
Freq: D, Name: TRANDATE, dtype: int64


In [22]:
trDF=DataFrame()                

In [23]:
trDF['date'] = trCountsChrono.index
trDF['transactions'] = trCountsChrono.values


In [24]:
trDF.columns

Index(['date', 'transactions'], dtype='object')

In [27]:
trDF.head()

,date,transactions
0,2009-01-01,176
1,2009-01-02,305
2,2009-01-03,365
3,2009-01-04,231
4,2009-01-05,144


In [28]:
trDF.dtypes

date            datetime64[ns]
transactions             int64
dtype: object

In [31]:
trMed=trDF.transactions.median()		# here's the median
trMed

136.0

In [32]:
heavyLight=lambda x  : x >= trMed and 'heavy' or 'light'  # an example anon function

In [33]:
trDF['vol']=trDF.transactions.map(heavyLight)	# 'vol' is the heavy/light column

In [34]:
trDF['monum']=trDF.date.dt.month	# .dt is the datetime accessor

In [35]:
trDFnd=trDF.drop('date',axis=1)	# axis=1 means here a column is selected to drop

In [36]:
trDFgrouped=trDFnd.groupby(['monum','vol']).sum()
trDFgrouped

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440
      light           564
8     heavy          1682
      light          1938
9     heavy          1921
      light          1942
10    heavy          2109
      light          2241
11    heavy          8402
      light            49
12    heavy         13168
      light           257

In [37]:
trDFgrouped.loc[11,'heavy']

transactions    8402
Name: (11, heavy), dtype: int64

In [38]:
 
trDFgrouped.loc[list(range(1,7))]

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495

In [39]:
trDFgrouped.iloc[0:6]		# .iloc here, but .loc above.

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664

In [40]:
trDFgrouped[(3,'light'):(7,'heavy')]

transactions
monum vol                
3     light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440

In [41]:
trDFgrouped[(3,'light'):6]

transactions
monum vol                
3     light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495

In [42]:
trDFgrouped.xs('light',level='vol')

,transactions
monum,
1,572
2,1625
3,1664
4,1727
5,2076
6,1495
7,564
8,1938
9,1942


In [43]:
trDFgrouped.xs('light',level='vol').T		# the transpose of the above

monum,1,2,3,4,5,6,7,8,9,10,11,12
transactions,572,1625,1664,1727,2076,1495,564,1938,1942,2241,49,257


In [44]:
mo=trDFgrouped.index.get_level_values(0) 	# the month numbers

In [46]:
volType=trDFgrouped.index.get_level_values(1)	# vol

In [47]:
trDFpiv=DataFrame({'month':mo,'vol': volType, 'transactions':trDFgrouped.transactions})		# data as a dict


In [49]:
trDFpived=trDFpiv.pivot(index='month',columns='vol',values='transactions')
trDFpived

vol,heavy,light
month,,
1,5255,572
2,761,1625
3,1130,1664
4,2327,1727
5,2172,2076
6,2878,1495
7,4440,564
8,1682,1938
9,1921,1942


In [50]:
trDFpiv['randy']=np.random.randn(len(trDFpiv))

In [52]:
trDFpived2=trDFpiv.pivot(index='month',columns='vol')
trDFpived2

transactions           randy          
vol          heavy light     heavy     light
month                                       
1             5255   572  0.035563  0.518436
2              761  1625  1.428370 -0.438097
3             1130  1664  0.334215 -0.256901
4             2327  1727 -0.519689 -0.000287
5             2172  2076  0.269636  0.459797
6             2878  1495  0.820173  1.035573
7             4440   564 -0.244989  0.358710
8             1682  1938  1.251150  2.043198
9             1921  1942 -1.256889  0.953022
10            2109  2241  0.094286  0.237398
11            8402    49 -2.185997  0.608476
12           13168   257  0.428843 -1.986141

In [ ]:
trDFpiv.drop('randy',axis=1,inplace=True)

In [55]:
trDFpiv

month    vol  transactions
monum vol                              
1     heavy      1  heavy          5255
      light      1  light           572
2     heavy      2  heavy           761
      light      2  light          1625
3     heavy      3  heavy          1130
      light      3  light          1664
4     heavy      4  heavy          2327
      light      4  light          1727
5     heavy      5  heavy          2172
      light      5  light          2076
6     heavy      6  heavy          2878
      light      6  light          1495
7     heavy      7  heavy          4440
      light      7  light           564
8     heavy      8  heavy          1682
      light      8  light          1938
9     heavy      9  heavy          1921
      light      9  light          1942
10    heavy     10  heavy          2109
      light     10  light          2241
11    heavy     11  heavy          8402
      light     11  light            49
12    heavy     12  heavy         13168
      light     12  light           257

In [60]:
xyzdata=xyzcustnew[['BUYER_STATUS','heavyCat','CHANNEL_ACQUISITION']]

In [61]:
xyzgrouped=xyzdata.groupby(['BUYER_STATUS','heavyCat','CHANNEL_ACQUISITION'])


In [62]:
xyzCountData = xyzgrouped.size()	# a MultiIndexed Series of counts

In [63]:
print(xyzCountData.unstack())

CHANNEL_ACQUISITION     CB    IB    RT
BUYER_STATUS heavyCat                 
ACTIVE       regular   443  1112  7393
             heavy     356   703  3325
INACTIVE     regular   691  1249  7056
             heavy       0     0     0
LAPSED       regular   372  1111  6368
             heavy       0     0     0


In [64]:
unStackxyz=xyzCountData.unstack()		# what we had just above

In [65]:
unStackxyz.T.stack()		# .T is the transpose 

BUYER_STATUS                  ACTIVE  INACTIVE  LAPSED
CHANNEL_ACQUISITION heavyCat                          
CB                  regular      443       691     372
                    heavy        356         0       0
IB                  regular     1112      1249    1111
                    heavy        703         0       0
RT                  regular     7393      7056    6368
                    heavy       3325         0       0

In [76]:
unStackxyz.T.stack(1)

BUYER_STATUS                  ACTIVE  INACTIVE  LAPSED
CHANNEL_ACQUISITION heavyCat                          
CB                  regular      443       691     372
                    heavy        356         0       0
IB                  regular     1112      1249    1111
                    heavy        703         0       0
RT                  regular     7393      7056    6368
                    heavy       3325         0       0

In [77]:
unStackxyz.T.stack(0).unstack(1)

heavyCat            regular                  heavy                
BUYER_STATUS         ACTIVE INACTIVE LAPSED ACTIVE INACTIVE LAPSED
CHANNEL_ACQUISITION                                               
CB                      443      691    372    356        0      0
IB                     1112     1249   1111    703        0      0
RT                     7393     7056   6368   3325        0      0

In [78]:
unStackxyz.T.stack(level=['heavyCat','BUYER_STATUS'])

CHANNEL_ACQUISITION  heavyCat  BUYER_STATUS
CB                   regular   ACTIVE           443
                               INACTIVE         691
                               LAPSED           372
                     heavy     ACTIVE           356
                               INACTIVE           0
                               LAPSED             0
IB                   regular   ACTIVE          1112
                               INACTIVE        1249
                               LAPSED          1111
                     heavy     ACTIVE           703
                               INACTIVE           0
                               LAPSED             0
RT                   regular   ACTIVE          7393
                               INACTIVE        7056
                               LAPSED          6368
                     heavy     ACTIVE          3325
                               INACTIVE           0
                               LAPSED             0
dtype: int64

In [79]:
unStackxyz.T.stack(level=['BUYER_STATUS','heavyCat'])

CHANNEL_ACQUISITION  BUYER_STATUS  heavyCat
CB                   ACTIVE        regular      443
                                   heavy        356
                     INACTIVE      regular      691
                                   heavy          0
                     LAPSED        regular      372
                                   heavy          0
IB                   ACTIVE        regular     1112
                                   heavy        703
                     INACTIVE      regular     1249
                                   heavy          0
                     LAPSED        regular     1111
                                   heavy          0
RT                   ACTIVE        regular     7393
                                   heavy       3325
                     INACTIVE      regular     7056
                                   heavy          0
                     LAPSED        regular     6368
                                   heavy          0
dtype: int64

In [80]:
xyzcust=xyzcustnew[['BUYER_STATUS','heavyCat','LTD_SALES']].copy()

In [81]:
xyzcustm=pd.melt(xyzcust,id_vars=['BUYER_STATUS','heavyCat'],var_name="LTD_SALES")


In [82]:
print(xyzcustm)

      BUYER_STATUS heavyCat  LTD_SALES   value
0         INACTIVE  regular  LTD_SALES    90.0
1           ACTIVE    heavy  LTD_SALES  4227.0
2           ACTIVE  regular  LTD_SALES   420.0
3         INACTIVE  regular  LTD_SALES  6552.0
4           ACTIVE  regular  LTD_SALES   189.0
...            ...      ...        ...     ...
30174       ACTIVE  regular  LTD_SALES  2736.0
30175       ACTIVE  regular  LTD_SALES  2412.0
30176     INACTIVE  regular  LTD_SALES   429.0
30177     INACTIVE  regular  LTD_SALES   651.0
30178       ACTIVE    heavy  LTD_SALES  4527.0

[30179 rows x 4 columns]


In [92]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS','heavyCat'],columns=['CHANNEL_ACQUISITION'])


CHANNEL_ACQUISITION             CB           IB           RT
BUYER_STATUS heavyCat                                       
ACTIVE       regular    205.334086   191.047662   167.993913
             heavy     2397.606742  1251.559033  1158.506165
INACTIVE     regular      0.000000     0.000000     0.000000
LAPSED       regular      0.000000     0.000000     0.000000

In [93]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'])


heavyCat                regular                                heavy  \
CHANNEL_ACQUISITION          CB          IB          RT           CB   
BUYER_STATUS                                                           
ACTIVE               205.334086  191.047662  167.993913  2397.606742   
INACTIVE               0.000000    0.000000    0.000000          NaN   
LAPSED                 0.000000    0.000000    0.000000          NaN   

heavyCat                                       
CHANNEL_ACQUISITION           IB           RT  
BUYER_STATUS                                   
ACTIVE               1251.559033  1158.506165  
INACTIVE                     NaN          NaN  
LAPSED                       NaN          NaN

In [94]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'],aggfunc=np.sum)


heavyCat             regular                          heavy            \
CHANNEL_ACQUISITION       CB        IB         RT        CB        IB   
BUYER_STATUS                                                            
ACTIVE               90963.0  212445.0  1241979.0  853548.0  879846.0   
INACTIVE                 0.0       0.0        0.0       NaN       NaN   
LAPSED                   0.0       0.0        0.0       NaN       NaN   

heavyCat                        
CHANNEL_ACQUISITION         RT  
BUYER_STATUS                    
ACTIVE               3852033.0  
INACTIVE                   NaN  
LAPSED                     NaN

In [95]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'],aggfunc=np.sum,margins=True)


heavyCat             regular                          heavy            \
CHANNEL_ACQUISITION       CB        IB         RT        CB        IB   
BUYER_STATUS                                                            
ACTIVE               90963.0  212445.0  1241979.0  853548.0  879846.0   
INACTIVE                 0.0       0.0        0.0       NaN       NaN   
LAPSED                   0.0       0.0        0.0       NaN       NaN   
All                  90963.0  212445.0  1241979.0  853548.0  879846.0   

heavyCat                              All  
CHANNEL_ACQUISITION         RT             
BUYER_STATUS                               
ACTIVE               3852033.0  7130814.0  
INACTIVE                   NaN        0.0  
LAPSED                     NaN        0.0  
All                  3852033.0  7130814.0

In [96]:
xyzGrouper=xyzcustnew.groupby(['BUYER_STATUS','heavyCat'])

In [97]:
xyzGrouper.agg({'YTD_SALES_2009': [np.mean, np.std],'LTD_SALES':[np.mean,np.std]})


YTD_SALES_2009                 LTD_SALES              
                                mean          std         mean           std
BUYER_STATUS heavyCat                                                       
ACTIVE       regular      172.707532   107.584023  1001.845105   1466.075631
             heavy       1274.048130  5434.616517  4096.179745  34210.646330
INACTIVE     regular        0.000000     0.000000   568.014784    850.966479
             heavy               NaN          NaN          NaN           NaN
LAPSED       regular        0.000000     0.000000   841.467329   1374.447756
             heavy               NaN          NaN          NaN           NaN

In [98]:
def coefV(x):                       # a baby CV function that accepts a sequence
    return np.std(x)/np.mean(x)


In [99]:
buyerStats=xyzcustnew[['BUYER_STATUS','LTD_SALES','LTD_TRANSACTIONS']]
buyerGrouper=buyerStats.groupby(['BUYER_STATUS'])
buyerGrouper.agg(coefV)

,LTD_SALES,LTD_TRANSACTIONS
BUYER_STATUS,,
ACTIVE,9.758480,1.153501
INACTIVE,1.498058,0.784441
LAPSED,1.633290,0.987139


In [100]:
def ptiles(x):
    p5=np.percentile(x,5)
    p95=np.percentile(x,95)
    return p5, p95


In [101]:
buyerGrouper.agg([np.mean, ptiles])

LTD_SALES                            LTD_TRANSACTIONS  \
                     mean                     ptiles             mean   
BUYER_STATUS                                                            
ACTIVE        2019.364086  (81.0, 6544.349999999997)         6.935794   
INACTIVE       568.014784             (60.0, 1776.0)         2.263895   
LAPSED         841.467329             (63.0, 2904.0)         3.498280   

                           
                   ptiles  
BUYER_STATUS               
ACTIVE        (1.0, 20.0)  
INACTIVE       (1.0, 6.0)  
LAPSED         (1.0, 9.0)

In [102]:
buyerGrouper.agg([np.mean,ptiles]).loc['ACTIVE','LTD_SALES']

mean                        2019.36
ptiles    (81.0, 6544.349999999997)
Name: ACTIVE, dtype: object

In [103]:
def IQR(x):
    p25=np.percentile(x,25)
    p75=np.percentile(x,75)
    return p75-p25

In [105]:
buyerGrouper.agg([np.mean, IQR])

LTD_SALES         LTD_TRANSACTIONS    
                     mean     IQR             mean IQR
BUYER_STATUS                                          
ACTIVE        2019.364086  1776.0         6.935794   7
INACTIVE       568.014784   492.0         2.263895   2
LAPSED         841.467329   772.5         3.498280   3

# Requirements :
1. Get the trDFgrouped data starting from the May heavy day counts to the August heavy counts
2. Group xyz customers using BUYER_STATUS, heavyCat, and ZIP, and apply np.sum function on the aggregated data for YTD_SALES_2009 and LTD_SALES columns

1. Get the trDFgrouped data starting from the May heavy day counts to the August heavy counts

In [109]:
trDFgrouped[(5,'heavy'):(8,'heavy')]

transactions
monum vol                
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440
      light           564
8     heavy          1682

2. Group xyz customers using BUYER_STATUS, heavyCat, and ZIP, and apply np.sum function on the aggregated data for YTD_SALES_2009 and LTD_SALES columns

In [129]:
ytd_ltd_sales = xyzcustnew.groupby(['BUYER_STATUS','heavyCat', 'ZIP'])
ytd_ltd_sales.agg({'YTD_SALES_2009': [np.sum], 'LTD_SALES':[np.sum]})

YTD_SALES_2009 LTD_SALES
                                       sum       sum
BUYER_STATUS heavyCat ZIP                           
ACTIVE       regular  0                NaN       NaN
                      60056        68913.0  332196.0
                      60060        68520.0  339567.0
                      60061        68328.0  400569.0
                      60062       141237.0  762387.0
...                                    ...       ...
LAPSED       heavy    60095            NaN       NaN
                      60096            NaN       NaN
                      60097            NaN       NaN
                      60098            NaN       NaN
                      60192            NaN       NaN

[222 rows x 2 columns]